<a href="https://colab.research.google.com/github/Sanazzzmi/hybrid-PSO_SVR/blob/main/hybrid_PSO_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Article idea](https://link.springer.com/content/pdf/10.1007/s11356-020-08792-3.pdf)

[Data](https://www.longpaddock.qld.gov.au/silo/point-data/)

### PSO_SVR

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#--- IMPORT DEPENDENCIES ------------------------------------------------------+

from __future__ import division
import random
import math
import copy    # array-copying convenience
import sys     # max float
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVR
import sklearn.model_selection
import numpy.random as rd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import numpy.random as rd

In [ ]:
#--- COST FUNCTION ------------------------------------------------------------+

# function we are attempting to optimize (minimize)


def func1(x):
    file_name='/content/drive/MyDrive/file_name2.csv'
    df=pd.read_csv(file_name)
    df = df.sort_index()
    y=df['et_short_crop']
    dff = df.drop('et_short_crop', 1)
    X=dff
    X_train,X_test,y_train,y_test= train_test_split(X, y, test_size = 0.2, random_state = 0)
    p0 = x[0]
    p1 = x[1]
    p2 = x[2]
    
    rbf_regressor = svm.SVR(kernel = 'rbf', C = p0 , epsilon =p1, gamma = p2 ).fit(X_train, y_train)  #svm        
    cv_accuracies = cross_val_score(rbf_regressor,X_test,y_test,cv =3,scoring = 'neg_mean_squared_error') # Taking negated value of MSE
    #To minimize the error rate
    accuracies = cv_accuracies.mean()            
    fitness_value = (1 - accuracies)*100
    
    return fitness_value


In [ ]:
#--- MAIN ---------------------------------------------------------------------+


class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.7       # constant inertia weight (how much to weigh the previous velocity)
        c1=2.1        # cognative constant
        c2=2.1        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
                
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        print ('position',pos_best_g)
        print ('error',err_best_g)

if __name__ == "__PSO__":
    main()

In [ ]:
#--- RUN ----------------------------------------------------------------------+

initial=[1,1,1]               # initial starting location [x1,x2...]
bounds=[(0.01,10),(0.01,10),(0.01,10)]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
PSO(func1,initial,bounds,num_particles=100,maxiter=50)

FINAL:
position [10, 0.5526287018532019, 0.01]
error 124.1202149570976


In [ ]:
#Apply Optimal Parameters to SVR
file_name='/content/drive/MyDrive/file_name2.csv'
df=pd.read_csv(file_name)
df = df.sort_index()
y=df['et_short_crop']
dff = df.drop('et_short_crop', 1)
X=dff
X_train,X_test,y_train,y_test= train_test_split(X, y, test_size = 0.2, random_state = 0)

svr_regressor= SVR(kernel='rbf', C = 10, epsilon = 0.5555153237418458,gamma=0.01)
svr_regressor.fit(X_train,y_train)

y_pred = svr_regressor.predict(X_test)

In [ ]:
# APPLYING K-FOLD CROSS VALIDATION on RF model
import numpy as np

accuracies = cross_val_score(svr_regressor, X = X_train, y = y_train, cv = 10)
accuracy_mean= accuracies.mean()
accuracies.std()*100
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE =", rmse)

RMSE = 0.4124281785351654
